In [2]:
!pip install biopython openpyxl

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/opt_einsum-3.4.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/nvfuser-0.2.23a0+6627725-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/dill-0.3.9-py3.12.egg is deprecated

In [2]:
import glob
import gzip
import json
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
from Bio import SeqIO
from pathlib import Path
from sklearn.metrics import roc_auc_score

from bionemo.noodles.nvfaidx import NvFaidx
from bionemo.testing.data.fasta import ALU_SEQUENCE, create_fasta_file


Outline:
1. Load evo2 model from hf checkpoints
1. Create fasta files for brca_1 data
2. Save to separate directory (one for reference, one for variance)
3. Use `predict` to score reference sequences
4. Use `predict` to score variant sequences
5. Compare (todo: ask John, my logic wrong for sure)
6. calculate auc
7. plot

## Load brca_1 data

In [3]:
brca1_df = pd.read_excel(
    os.path.join('brca1', '41586_2018_461_MOESM3_ESM.xlsx'),
    header=2,
)
brca1_df = brca1_df[[
    'chromosome', 'position (hg19)', 'reference', 'alt', 'function.score.mean', 'func.class',
]]

# Rename columns
brca1_df.rename(columns={
    'chromosome': 'chrom',
    'position (hg19)': 'pos',
    'reference': 'ref',
    'alt': 'alt',
    'function.score.mean': 'score',
    'func.class': 'class',
}, inplace=True)

# Convert to two-class system
brca1_df['class'] = brca1_df['class'].replace(['FUNC', 'INT'], 'FUNC/INT')

brca1_df.head(10)

,chrom,pos,ref,alt,score,class
0,17,41276135,T,G,-0.372611,FUNC/INT
1,17,41276135,T,C,-0.045313,FUNC/INT
2,17,41276135,T,A,-0.108254,FUNC/INT
3,17,41276134,T,G,-0.277963,FUNC/INT
4,17,41276134,T,C,-0.388414,FUNC/INT
5,17,41276134,T,A,-0.280973,FUNC/INT
6,17,41276133,C,T,-0.973683,FUNC/INT
7,17,41276133,C,G,-0.373489,FUNC/INT
8,17,41276133,C,A,0.006314,FUNC/INT
9,17,41276132,A,T,-0.207552,FUNC/INT


In [4]:
WINDOW_SIZE = 8192

# Read the reference genome sequence of chromosome 17
with gzip.open(os.path.join('brca1', 'GRCh37.p13_chr17.fna.gz'), "rt") as handle:
    for record in SeqIO.parse(handle, "fasta"):
        seq_chr17 = str(record.seq)
        break

def parse_sequences(pos, ref, alt):
    """
    Parse reference and variant sequences from the reference genome sequence.
    """
    p = pos - 1 # Convert to 0-indexed position
    full_seq = seq_chr17

    ref_seq_start = max(0, p - WINDOW_SIZE//2)
    ref_seq_end = min(len(full_seq), p + WINDOW_SIZE//2)
    ref_seq = seq_chr17[ref_seq_start:ref_seq_end]
    snv_pos_in_ref = min(WINDOW_SIZE//2, p)
    var_seq = ref_seq[:snv_pos_in_ref] + alt + ref_seq[snv_pos_in_ref+1:]

    # Sanity checks
    assert len(var_seq) == len(ref_seq)
    assert ref_seq[snv_pos_in_ref] == ref
    assert var_seq[snv_pos_in_ref] == alt

    return ref_seq, var_seq

## Load Evo2

We'll load evo2 weights from hugging face.



In [7]:
!python /workspace/bionemo2/sub-packages/bionemo-evo2/src/bionemo/evo2/utils/checkpoint/convert_to_nemo.py --model-path hf://arcinstitute/savanna_evo2_1b_base --model-size 1b --output-dir nemo2_evo2_1b_8k

[WARNING  | bitsandbytes.cextension]: Could not find the bitsandbytes CUDA binary at PosixPath('/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda128.so')
[WARNING  | bitsandbytes.cextension]: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
[NeMo W 2025-02-25 06:40:57 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[WARNING  | py.warnings        ]: /usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/selective_scan_interface.py:163: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd

[WARNING  | py.warnings        ]: /usr/local/lib/python3.1

We need to store a `Path` reference to where these checkpoints were saved so we can use them in the `predict` command later:

In [5]:
# todo: match path here with above command via shared string template
checkpoint_path = Path("nemo2_evo2_1b_8k")

### Save data to directories

Convert both the reference and variant fasta sequences to their own respective `.fasta` files.

In [6]:
# Create output directory
output_dir = Path("brca1_fasta_files")
output_dir.mkdir(parents=True, exist_ok=True)

# Save reference sequences to FASTA
ref_fasta_path = output_dir / "brca1_reference_sequences.fasta"
with open(ref_fasta_path, "w") as f:
    for idx, row in brca1_df.iterrows():
        ref_seq, _ = parse_sequences(row['pos'], row['ref'], row['alt'])
        # Create unique header for each sequence
        f.write(f">BRCA1_ref_{idx}_pos_{row['pos']}_{row['ref']}_class_{row['class']}\n")
        f.write(f"{ref_seq}\n")

# Save variant sequences to FASTA
var_fasta_path = output_dir / "brca1_variant_sequences.fasta"
with open(var_fasta_path, "w") as f:
    for idx, row in brca1_df.iterrows():
        _, var_seq = parse_sequences(row['pos'], row['ref'], row['alt'])
        # Create unique header for each sequence
        f.write(f">BRCA1_var_{idx}_pos_{row['pos']}_{row['ref']}to{row['alt']}_class_{row['class']}\n")
        f.write(f"{var_seq}\n")

print(f"Reference sequences saved to: {ref_fasta_path}")
print(f"Variant sequences saved to: {var_fasta_path}")

Reference sequences saved to: brca1_fasta_files/brca1_reference_sequences.fasta
Variant sequences saved to: brca1_fasta_files/brca1_variant_sequences.fasta


### Generate Samples of Reference/Variants.

The full samples takes too long for debugging

In [94]:
brca1_df.shape

(3893, 6)

In [1]:
# Create a test directory
test_output_dir = Path("brca1_test_output")
test_output_dir.mkdir(parents=True, exist_ok=True)

test_ref_dir = test_output_dir / "reference"
test_var_dir = test_output_dir / "variant"
test_ref_dir.mkdir(parents=True, exist_ok=True)
test_var_dir.mkdir(parents=True, exist_ok=True)

num_samples = 3000

# Get indices for balanced samples
func_int_indices = brca1_df[brca1_df['class'] == 'FUNC/INT'].index[:num_samples//2].tolist()
lof_indices = brca1_df[brca1_df['class'] == 'LOF'].index[:num_samples//2].tolist()
balanced_indices = func_int_indices + lof_indices

# Save reference sequences to test FASTA
test_ref_fasta_path = test_output_dir / "test_reference_sequences.fasta"
with open(test_ref_fasta_path, "w") as f:
    for idx in balanced_indices:
        row = brca1_df.loc[idx]
        ref_seq, _ = parse_sequences(row['pos'], row['ref'], row['alt'])
        f.write(f">BRCA1_ref_{idx}_pos_{row['pos']}_{row['ref']}_class_{row['class']}\n")
        f.write(f"{ref_seq}\n")

# Save variant sequences to test FASTA
test_var_fasta_path = test_output_dir / "test_variant_sequences.fasta"
with open(test_var_fasta_path, "w") as f:
    for idx in balanced_indices:
        row = brca1_df.loc[idx]
        _, var_seq = parse_sequences(row['pos'], row['ref'], row['alt'])
        f.write(f">BRCA1_var_{idx}_pos_{row['pos']}_{row['ref']}to{row['alt']}_class_{row['class']}\n")
        f.write(f"{var_seq}\n")

print(f"Test reference sequences saved to: {test_ref_fasta_path}")
print(f"Test variant sequences saved to: {test_var_fasta_path}")
print(f"Sample distribution: {len(func_int_indices)} FUNC/INT, {len(lof_indices)} LOF")

# Create test predict commands
test_predict_ref_command = (
    f"predict_evo2 --fasta {test_ref_fasta_path} --ckpt-dir {checkpoint_path} "
    f"--output-dir {test_ref_dir} --model-size 1b --tensor-parallel-size 1 "
    "--pipeline-model-parallel-size 1 --context-parallel-size 1"
)

test_predict_var_command = (
    f"predict_evo2 --fasta {test_var_fasta_path} --ckpt-dir {checkpoint_path} "
    f"--output-dir {test_var_dir} --model-size 1b --tensor-parallel-size 1 "
    "--pipeline-model-parallel-size 1 --context-parallel-size 1"
)


NameError: name 'Path' is not defined

### Predict reference seqs (sample)


In [96]:
!{test_predict_ref_command}

[WARNING  | bitsandbytes.cextension]: Could not find the bitsandbytes CUDA binary at PosixPath('/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda128.so')
[WARNING  | bitsandbytes.cextension]: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
[NeMo W 2025-02-26 07:02:04 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[WARNING  | py.warnings        ]: /usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/selective_scan_interface.py:163: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd

[WARNING  | py.warnings        ]: /usr/local/lib/python3.1

: 

### Predict variant seqs (sample)

In [ ]:
!{test_predict_var_command}

[WARNING  | bitsandbytes.cextension]: Could not find the bitsandbytes CUDA binary at PosixPath('/usr/local/lib/python3.12/dist-packages/bitsandbytes/libbitsandbytes_cuda128.so')
[WARNING  | bitsandbytes.cextension]: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
[NeMo W 2025-02-26 06:35:37 nemo_logging:405] /usr/local/lib/python3.12/dist-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
    
[WARNING  | py.warnings        ]: /usr/local/lib/python3.12/dist-packages/mamba_ssm/ops/selective_scan_interface.py:163: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd

[WARNING  | py.warnings        ]: /usr/local/lib/python3.1

Ok, so far, we've turned the sequences into fasta files with unique headers per sequence.

Then, we ran predictions for these sequences.

Recall, we created seq_idx_maps that map FASTA sequence IDs (e.g. the sequence headers) to indices in the prediction arrays.

so, now we need to

1. load the prediction files and corresponding sequence id maps
2. map each sequence id to its original dataframe index
3. calculate the delta scores

So, let's load the prediction files and sequence id maps:

In [ ]:
# Find and load prediction files
ref_pred_files = glob.glob(os.path.join(test_ref_dir, "predictions__rank_*.pt"))
var_pred_files = glob.glob(os.path.join(test_var_dir, "predictions__rank_*.pt"))

# Load sequence ID maps (maps sequence ID -> prediction index)
with open(os.path.join(test_ref_dir, "seq_idx_map.json"), "r") as f:
    ref_seq_idx_map = json.load(f)
with open(os.path.join(test_var_dir, "seq_idx_map.json"), "r") as f:
    var_seq_idx_map = json.load(f)

# Load predictions
ref_preds = torch.load(ref_pred_files[0])
var_preds = torch.load(var_pred_files[0])

### Joining Everything Together

(TODO: remove from final nb)

Enumerating here to make sure it's correct:

1. **Original Dataframe**: The original dataframe of genetic variants. 
    - We can get uuids via a combination of the row `idx` and columns `chrom`, `pos`, `ref`, `alt`.

2. **Fasta Sequences**: When we convert the above to reference and variant sequences, we can embed the above uuid/row-index into the header:
    - e.g. for each sequence do: `>BRCA1_ref_{idx}_pos_{row['pos']}_{row['ref']}_class_{row['class']}`, so we can map back to the dataframe

3. **Predictions**: Predictions generate logits and a mapping file, `seq_idx_map.json`.
    - `seq_idx_map.json` maps FASTA headers -> indices in the prediction array. So, for a given sequence, we can find the index of the prediction that corresponds to it.


So now we can map original dataframe -> sequence headers -> predicitons

### Map scores back to the original dataframe

Create the mappings between the prediction scores and the original dataframe indices. First, we extract the dataframe index from each FASTA sequence ID by parsing the ID string. Then, we create two dictionaries that map from dataframe indices to prediction scores: one for reference sequences and one for variant sequences. Finally, we verify our mappings by checking how many sequences have both reference and variant scores.

In [ ]:
# Create mapping from sequence ID to original dataframe index
ref_df_indices = {}  # Maps: sequence ID -> dataframe index
var_df_indices = {}  # Maps: sequence ID -> dataframe index

# populate dicts mapping sequence IDs -> df row index
# Extract dataframe indices from sequence IDs
for seq_id in ref_seq_idx_map.keys():
    parts = seq_id.split('_')
    try:
        df_idx = int(parts[2])  # Get index from "BRCA1_ref_INDEX_..."
        ref_df_indices[seq_id] = df_idx
    except (IndexError, ValueError):
        print(f"Warning: Could not parse index from reference ID: {seq_id}")

for seq_id in var_seq_idx_map.keys():
    parts = seq_id.split('_')
    try:
        df_idx = int(parts[2])  # Get index from "BRCA1_var_INDEX_..."
        var_df_indices[seq_id] = df_idx
    except (IndexError, ValueError):
        print(f"Warning: Could not parse index from variant ID: {seq_id}")

In [ ]:

# Create maps from dataframe index to prediction scores

# map dataframe index to predictions
# find sequence id in ref_df_indices
# use this to get the df row index
# then use this to map the predictions to the correct df row index
df_to_ref_score = {}  # Maps: dataframe index -> reference score
df_to_var_score = {}  # Maps: dataframe index -> variant score

# Map dataframe indices to scores using the sequence ID maps
for seq_id, pred_idx in ref_seq_idx_map.items():
    if seq_id in ref_df_indices: # if keys match
        df_idx = ref_df_indices[seq_id]
        df_to_ref_score[df_idx] = ref_preds['token_logits'][pred_idx]

for seq_id, pred_idx in var_seq_idx_map.items():
    if seq_id in var_df_indices:
        df_idx = var_df_indices[seq_id]
        df_to_var_score[df_idx] = var_preds['token_logits'][pred_idx]

# Verification: How many sequences could we map?
common_indices = set(df_to_ref_score.keys()).intersection(set(df_to_var_score.keys()))
print(f"\nSuccessfully mapped scores for {len(common_indices)} sequences")


Successfully mapped scores for 1600 sequences


In [ ]:
# Create a validation dataframe that includes prediction arrays
validation_rows = []

# Process each dataframe index with both reference and variant data
for df_idx in common_indices:
    # Get original data
    row_data = brca1_df.loc[df_idx].to_dict()
    
    # Find the corresponding sequence IDs
    ref_seq_id = next((seq_id for seq_id, i in ref_df_indices.items() if i == df_idx), None)
    var_seq_id = next((seq_id for seq_id, i in var_df_indices.items() if i == df_idx), None)
    
    if ref_seq_id and var_seq_id:
        # Get prediction indices
        ref_pred_idx = ref_seq_idx_map[ref_seq_id]
        var_pred_idx = var_seq_idx_map[var_seq_id]
        
        # Get prediction vectors
        ref_prediction = ref_preds["token_logits"][ref_pred_idx]
        var_prediction = var_preds["token_logits"][var_pred_idx]
        
        # Create a row for the validation dataframe
        validation_row = {
            'df_idx': df_idx,
            'chrom': row_data['chrom'],
            'pos': row_data['pos'],
            'ref': row_data['ref'],
            'alt': row_data['alt'],
            'score': row_data['score'],
            'class': row_data['class'],
            'ref_fasta_header': ref_seq_id,
            'var_fasta_header': var_seq_id,
            'ref_pred_idx': ref_pred_idx,
            'var_pred_idx': var_pred_idx,
            'ref_prediction': ref_prediction,
            'var_prediction': var_prediction
        }
        
        validation_rows.append(validation_row)

# Create the validation dataframe
validation_df = pd.DataFrame(validation_rows)

# Print summary
print(f"Created validation dataframe with {len(validation_df)} rows")
print(f"Number of unique dataframe indices: {validation_df['df_idx'].nunique()}")

validation_df.head()

Created validation dataframe with 1600 rows
Number of unique dataframe indices: 1600


,df_idx,chrom,pos,ref,alt,score,class,ref_fasta_header,var_fasta_header,ref_pred_idx,var_pred_idx,ref_prediction,var_prediction
0,0,17,41276135,T,G,-0.372611,FUNC/INT,BRCA1_ref_0_pos_41276135_T_class_FUNC/INT,BRCA1_var_0_pos_41276135_TtoG_class_FUNC/INT,268,33,"[[tensor(-7.6875), tensor(-47.5000), tensor(-4...","[[tensor(-7.7188), tensor(-47.5000), tensor(-4..."
1,1,17,41276135,T,C,-0.045313,FUNC/INT,BRCA1_ref_1_pos_41276135_T_class_FUNC/INT,BRCA1_var_1_pos_41276135_TtoC_class_FUNC/INT,1009,273,"[[tensor(-7.6875), tensor(-47.5000), tensor(-4...","[[tensor(-7.7188), tensor(-47.5000), tensor(-4..."
2,2,17,41276135,T,A,-0.108254,FUNC/INT,BRCA1_ref_2_pos_41276135_T_class_FUNC/INT,BRCA1_var_2_pos_41276135_TtoA_class_FUNC/INT,446,1179,"[[tensor(-7.6875), tensor(-47.5000), tensor(-4...","[[tensor(-7.6875), tensor(-47.5000), tensor(-4..."
3,3,17,41276134,T,G,-0.277963,FUNC/INT,BRCA1_ref_3_pos_41276134_T_class_FUNC/INT,BRCA1_var_3_pos_41276134_TtoG_class_FUNC/INT,822,549,"[[tensor(-7.7188), tensor(-47.5000), tensor(-4...","[[tensor(-7.6875), tensor(-47.5000), tensor(-4..."
4,4,17,41276134,T,C,-0.388414,FUNC/INT,BRCA1_ref_4_pos_41276134_T_class_FUNC/INT,BRCA1_var_4_pos_41276134_TtoC_class_FUNC/INT,968,175,"[[tensor(-7.7188), tensor(-47.5000), tensor(-4...","[[tensor(-7.7188), tensor(-47.5000), tensor(-4..."


# Now, Score

1. Apply log_softmax to renormalize the vector.

That is, each prediction is shape [8192 x 512]. We want to normalize across this vocab dimension (512) so that the probabilities represent the most likely nucleotide base at that index in the sequence.

In [ ]:
# Step 1: Apply log_softmax to renormalize the vectors - CORRECTED
processed_data = []

for _, row in validation_df.iterrows():
    # Get prediction tensors
    ref_logits = row['ref_prediction']
    var_logits = row['var_prediction']
    
    # Check tensor dimensions
    print(f"Raw logits shape: {ref_logits.shape}")
    
    # Apply log_softmax along the covab, so 
    # bc shape is [sequence_length, vocab_size], use dim=1
    if len(ref_logits.shape) == 2:
        ref_log_probs = torch.log_softmax(ref_logits, dim=1)
        var_log_probs = torch.log_softmax(var_logits, dim=1)
    else:  # 3D tensor with batch dimension
        ref_log_probs = torch.log_softmax(ref_logits, dim=2)
        var_log_probs = torch.log_softmax(var_logits, dim=2)
    
    # Store in processed data
    new_row = {
        'df_idx': row['df_idx'],
        'ref_fasta_header': row['ref_fasta_header'],
        'var_fasta_header': row['var_fasta_header'],
        'ref_pred_idx': row['ref_pred_idx'],
        'var_pred_idx': row['var_pred_idx'],
        'class': row['class'],
        'ref_log_probs': ref_log_probs,
        'var_log_probs': var_log_probs
    }
    
    processed_data.append(new_row)

print(f"Processed {len(processed_data)} sequences")

# Verification: Check that exponential sum equals 1 for a sample of positions
ref_log_probs = processed_data[0]['ref_log_probs']
var_log_probs = processed_data[0]['var_log_probs']

print(f"Log probs shape after correction: {ref_log_probs.shape}")

# Check 3 random positions in the sequence
positions_to_check = [0, ref_log_probs.shape[0]//2, ref_log_probs.shape[0]-1]
for pos in positions_to_check:
    # For 2D tensor [seq_len, vocab_size], sum along dimension 1
    if len(ref_log_probs.shape) == 2:
        ref_sum = torch.exp(ref_log_probs[pos]).sum().item()
        var_sum = torch.exp(var_log_probs[pos]).sum().item()
    else:  # For 3D tensor [batch, seq_len, vocab_size], sum along dimension 2
        ref_sum = torch.exp(ref_log_probs[0, pos]).sum().item()
        var_sum = torch.exp(var_log_probs[0, pos]).sum().item()
        
    print(f"Position {pos}:")
    print(f"  Reference exp(log_probs) sum: {ref_sum:.8f}")
    print(f"  Variant exp(log_probs) sum: {var_sum:.8f}")

Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192, 512])
Raw logits shape: torch.Size([8192

In [ ]:
# check normalization (log to sum the exp)
torch.exp(processed_data[0]['ref_log_probs'][1]).sum().item()

1.0

In [ ]:
ref_preds['token_logits'][0].shape

torch.Size([8192, 512])

Now, in order to identify the likelihood of the sequence, we need to find the corresponding tokens to compare their probabilities. More specifically, we have, for each prediction, a tensor of [8192 x 512], which represents the model's predicted log probabilities for all possible tokens (all 512 of them) at each position in the sequence (all 8192 positions). But what we want is to extract only the probabilities of the tokens that actually appeared in our original sequence, resulting in a tensor of [8192 x 1]. This [8192 x 1] tensor represents the log probability of each actual token given all the previous tokens (causal likelihood). Once we have these log probabilities for the actual tokens, we can average them to get a single score that represents how "natural" the entire sequence is according to the model.

In [ ]:
# Step 2: Pick the tokens by reading the original sequences
import re

# Function to read sequence from FASTA file
def read_sequence_from_fasta(fasta_header, fasta_file_path):
    """Read the sequence for a specific header from a FASTA file."""
    with open(fasta_file_path, 'r') as f:
        current_header = None
        sequence = ""
        found = False
        
        for line in f:
            line = line.strip()
            if line.startswith('>'):
                # This is a header line
                if found and current_header == fasta_header:
                    break
                current_header = line[1:]  # Remove the '>'
                if current_header == fasta_header:
                    found = True
            elif found:
                # This is a sequence line for our target header
                sequence += line
        
        if found:
            return sequence
        else:
            raise ValueError(f"Header '{fasta_header}' not found in FASTA file")

# Process each entry to get token IDs
for i, entry in enumerate(processed_data):
    # Get FASTA headers
    ref_header = entry['ref_fasta_header']
    var_header = entry['var_fasta_header']
    
    # Read sequences from FASTA files
    ref_seq = read_sequence_from_fasta(ref_header, test_ref_fasta_path)
    var_seq = read_sequence_from_fasta(var_header, test_var_fasta_path)
    
    # Convert sequences to token IDs (simple byte-level tokenization)
    ref_tokens = torch.tensor([ord(c) for c in ref_seq])
    var_tokens = torch.tensor([ord(c) for c in var_seq])
    
    # Store token IDs in our processed data
    processed_data[i]['ref_tokens'] = ref_tokens
    processed_data[i]['var_tokens'] = var_tokens

# Verify tokens by checking a few examples
example_idx = 0  # Check the first entry
ref_seq_sample = read_sequence_from_fasta(processed_data[example_idx]['ref_fasta_header'], test_ref_fasta_path)
ref_tokens_sample = processed_data[example_idx]['ref_tokens']

print(f"First 50 characters of reference sequence: {ref_seq_sample[:50]}")
print(f"First 10 token IDs: {ref_tokens_sample[:10].tolist()}")
print(f"Token length: {len(ref_tokens_sample)}")
print(f"Sequence length: {len(ref_seq_sample)}")

First 50 characters of reference sequence: TTAAACACTTTTCAAACCAGGCAATATTTTAGGCCTACTGTATATTTGCA
First 10 token IDs: [84, 84, 65, 65, 65, 67, 65, 67, 84, 84]
Token length: 8192
Sequence length: 8192


In [ ]:
# pd.DataFrame(processed_data)

In [ ]:
# double check that it's off by one (the swap)
(processed_data[0]['ref_tokens'] == processed_data[0]['var_tokens']).sum() == WINDOW_SIZE - 1

tensor(True)

In [ ]:
processed_data[0]['ref_log_probs']

tensor([[ -9.9760, -49.7885, -49.7885,  ..., -49.7885, -49.7885, -49.7885],
        [ -8.3141, -49.7516, -49.7516,  ..., -49.7516, -49.7516, -49.7516],
        [ -8.2281, -50.0094, -50.0094,  ..., -49.7594, -49.7594, -49.7594],
        ...,
        [-10.6458, -37.8333, -37.8333,  ..., -37.8333, -37.8333, -37.8333],
        [-11.3194, -41.4444, -41.4444,  ..., -41.6944, -41.4444, -41.4444],
        [-11.7649, -44.7649, -44.7649,  ..., -44.7649, -44.7649, -44.7649]])

Now that we have the tokens, we need to use them to extract the log probabilities of each actual token from our normalized prediction tensors. This will give us the log probabilities of the sequence, position by position.

In [ ]:
ref_log_probs = processed_data[0]['ref_log_probs']
ref_tokens =    processed_data[0]['ref_tokens']
print(ref_log_probs.shape)
print(ref_tokens.shape)

ref_input_tokens = ref_tokens[:-1]
ref_target_tokens = ref_tokens[1:]
print(ref_tokens)
print(ref_input_tokens)
print(ref_target_tokens)

# also shift log probs
ref_log_probs_shifted = ref_log_probs[:-1]
print(ref_log_probs_shifted.shape)

torch.Size([8192, 512])
torch.Size([8192])
tensor([84, 84, 65,  ..., 84, 67, 65])
tensor([84, 84, 65,  ..., 84, 84, 67])
tensor([84, 65, 65,  ..., 84, 67, 65])
torch.Size([8191, 512])


In [ ]:
ref_token_log_probs = []

print(ref_input_tokens.shape)
# for each input token
for j in range(len(ref_input_tokens)):
    # find the target token
    ref_target = ref_target_tokens[j].item()
    print('ref_target', ref_target)
    print(ref_log_probs_shifted[j, ref_target])
    # the log probs is [8191 x 512]: get the current seq element (j)'s value for the given nucleotide (ref_target)
    ref_token_log_probs.append(ref_log_probs_shifted[j, ref_target].item())
print(ref_token_log_probs)

torch.Size([8191])
ref_target 84
tensor(-1.3706)
ref_target 65
tensor(-1.5446)
ref_target 65
tensor(-1.1813)
ref_target 65
tensor(-1.0771)
ref_target 67
tensor(-1.7918)
ref_target 65
tensor(-1.0458)
ref_target 67
tensor(-1.7176)
ref_target 84
tensor(-1.2249)
ref_target 84
tensor(-1.0615)
ref_target 84
tensor(-1.0336)
ref_target 84
tensor(-1.0575)
ref_target 67
tensor(-1.7532)
ref_target 65
tensor(-1.3698)
ref_target 65
tensor(-1.0697)
ref_target 65
tensor(-0.9467)
ref_target 67
tensor(-1.7666)
ref_target 67
tensor(-1.6703)
ref_target 65
tensor(-0.9896)
ref_target 71
tensor(-1.7529)
ref_target 71
tensor(-1.7619)
ref_target 67
tensor(-1.4810)
ref_target 65
tensor(-1.0880)
ref_target 65
tensor(-1.1434)
ref_target 84
tensor(-1.3570)
ref_target 65
tensor(-1.2701)
ref_target 84
tensor(-1.1750)
ref_target 84
tensor(-1.0452)
ref_target 84
tensor(-0.8757)
ref_target 84
tensor(-0.9264)
ref_target 65
tensor(-1.3956)
ref_target 71
tensor(-1.8795)
ref_target 71
tensor(-1.7315)
ref_target 67
tensor(

In [ ]:
# Step 3: Extract log probabilities for the actual tokens
for i, entry in enumerate(processed_data):
    # Get the log probability distributions and token IDs
    ref_log_probs = entry['ref_log_probs']
    var_log_probs = entry['var_log_probs']
    ref_tokens = entry['ref_tokens']
    var_tokens = entry['var_tokens']
    
    # Ensure tokens are of appropriate length for the log_probs
    # seq_length = ref_log_probs.shape[0]
    # ref_tokens = ref_tokens[:seq_length]
    # var_tokens = var_tokens[:seq_length]
    
    # Shift tokens to get targets (we want to predict token at position i+1 given tokens up to i)
    # Remove the last token from input and first token from targets
    ref_input_tokens = ref_tokens[:-1]
    var_input_tokens = var_tokens[:-1]
    ref_target_tokens = ref_tokens[1:]
    var_target_tokens = var_tokens[1:]
    
    # Shift log_probs similarly (remove last prediction)
    ref_log_probs_shifted = ref_log_probs[:-1]
    var_log_probs_shifted = var_log_probs[:-1]
    
    # Extract the log probability for each actual token
    ref_token_log_probs = []
    var_token_log_probs = []
    
    # For each position, extract the log probability of the actual next token
    for j in range(len(ref_input_tokens)):
        # Get the target token at this position
        ref_target = ref_target_tokens[j].item()
        var_target = var_target_tokens[j].item()
        
        # Ensure token is within bounds of vocabulary
        if ref_target < ref_log_probs_shifted.shape[1] and var_target < var_log_probs_shifted.shape[1]:
            # Extract log probability for this token
            ref_token_log_probs.append(ref_log_probs_shifted[j, ref_target].item())
            var_token_log_probs.append(var_log_probs_shifted[j, var_target].item())
    
    # Convert to tensors
    ref_token_log_probs = torch.tensor(ref_token_log_probs)
    var_token_log_probs = torch.tensor(var_token_log_probs)
    
    # Store in our processed data
    processed_data[i]['ref_token_log_probs'] = ref_token_log_probs
    processed_data[i]['var_token_log_probs'] = var_token_log_probs

# Verify extraction of token log probabilities
example_idx = 0
ref_token_log_probs = processed_data[example_idx]['ref_token_log_probs']
var_token_log_probs = processed_data[example_idx]['var_token_log_probs']

print(f"Extracted {len(ref_token_log_probs)} token log probabilities for reference sequence")
print(f"First 5 reference token log probabilities: {ref_token_log_probs[:5].tolist()}")
print(f"First 5 variant token log probabilities: {var_token_log_probs[:5].tolist()}")

Extracted 8191 token log probabilities for reference sequence
First 5 reference token log probabilities: [-1.3705531358718872, -1.5445784330368042, -1.1812506914138794, -1.0771363973617554, -1.791759729385376]
First 5 variant token log probabilities: [-1.3705517053604126, -1.5415070056915283, -1.1812506914138794, -1.0747318267822266, -1.7890199422836304]


So now, we have two vectors: one for ref and one for var, of the log seq probabilities.

Now, we calculate the mean of these, then subtract them from each other.

In [ ]:
# Step 4: Calculate mean log probability (log_p_seq_mu) for each sequence
for i, entry in enumerate(processed_data):
    # Get token log probabilities
    ref_token_log_probs = entry['ref_token_log_probs']
    var_token_log_probs = entry['var_token_log_probs']
    
    # Calculate mean log probability
    ref_mean_log_prob = ref_token_log_probs.mean().item()
    var_mean_log_prob = var_token_log_probs.mean().item()
    
    # Calculate delta score (variant - reference)
    delta_score = var_mean_log_prob - ref_mean_log_prob
    
    # Store scores
    processed_data[i]['ref_score'] = ref_mean_log_prob
    processed_data[i]['var_score'] = var_mean_log_prob
    processed_data[i]['delta_score'] = delta_score

# Convert processed data to a DataFrame for easier analysis
results_df = pd.DataFrame([{
    'df_idx': entry['df_idx'],
    'class': entry['class'],
    'ref_score': entry['ref_score'],
    'var_score': entry['var_score'],
    'delta_score': entry['delta_score']
} for entry in processed_data])

results_df

,df_idx,class,ref_score,var_score,delta_score
0,0,FUNC/INT,-1.080265,-1.080446,-0.000181
1,1,FUNC/INT,-1.080265,-1.080169,0.000095
2,2,FUNC/INT,-1.080265,-1.080053,0.000212
3,3,FUNC/INT,-1.079231,-1.079564,-0.000333
4,4,FUNC/INT,-1.079231,-1.079340,-0.000109
...,...,...,...,...,...
1595,3756,LOF,-1.101452,-1.101409,0.000043
1596,3758,LOF,-1.103631,-1.103389,0.000242
1597,3761,LOF,-1.105333,-1.105293,0.000040
1598,3762,LOF,-1.105333,-1.105511,-0.000179


In [ ]:
# Display summary statistics
print("\nSummary statistics:")
print(f"Reference scores: mean={results_df['ref_score'].mean():.6f}, std={results_df['ref_score'].std():.6f}")
print(f"Variant scores: mean={results_df['var_score'].mean():.6f}, std={results_df['var_score'].std():.6f}")
print(f"Delta scores: mean={results_df['delta_score'].mean():.6f}, std={results_df['delta_score'].std():.6f}")

# Display scores by class
print("\nScores by class:")
print(results_df.groupby('class')['delta_score'].describe())

# Calculate AUROC if both classes are present
if len(results_df['class'].unique()) > 1:
    from sklearn.metrics import roc_auc_score
    y_true = (results_df['class'] == 'LOF').astype(int)
    auroc = roc_auc_score(y_true, -results_df['delta_score'])  # Negative because lower delta = more likely disruptive
    print(f"\nAUROC: {auroc:.4f}")


Summary statistics:
Reference scores: mean=-0.967987, std=0.082831
Variant scores: mean=-0.968069, std=0.082824
Delta scores: mean=-0.000082, std=0.000281

Scores by class:
          count      mean       std       min       25%       50%       75%  \
class                                                                         
FUNC/INT  800.0 -0.000064  0.000279 -0.001442 -0.000211 -0.000061  0.000092   
LOF       800.0 -0.000100  0.000281 -0.001063 -0.000284 -0.000075  0.000099   

               max  
class               
FUNC/INT  0.000916  
LOF       0.000799  

AUROC: 0.5294


In [ ]:
4

### **BUG I THINK** Step 2. Calculate Sequence Liklihoods

calculate the normalized log-likelihood for each sequence. For each position in the sequence, we compute the probability the model assigns to the next token, then sum these log probabilities and normalize by sequence length. This gives us a measure of how "natural" or "expected" the sequence is according to the model. We calculate these scores for both reference and variant sequences.

In [ ]:
ref_preds["token_logits"][0].shape

In [ ]:
ref_preds["pad_mask"][0].shape

In [ ]:
# Define function to calculate sequence log-likelihood
def calculate_sequence_likelihood(token_logits, pad_mask):
    """Calculate normalized log-likelihood for sequences."""
    # Convert to log probabilities
    # token_logits is [bs x seq_len x vocab_size]
    # need probs for vocab, so take dim=-1
    log_probs = torch.log_softmax(token_logits, dim=-1)
    
    # Shift sequences to predict next token
    input_ids = torch.argmax(token_logits, dim=-1)
    next_tokens = input_ids[:, 1:].contiguous()
    shifted_logits = log_probs[:, :-1, :].contiguous()
    shifted_mask = pad_mask[:, 1:].contiguous()
    
    # Get log probabilities for actual next tokens
    gathered_log_probs = torch.gather(
        shifted_logits, 
        dim=2, 
        index=next_tokens.unsqueeze(-1)
    ).squeeze(-1)
    
    # Sum log probabilities and normalize
    masked_log_probs = gathered_log_probs * shifted_mask
    seq_log_likelihood = masked_log_probs.sum(dim=1)
    seq_length = shifted_mask.sum(dim=1)
    normalized_log_likelihood = seq_log_likelihood / seq_length
    
    return normalized_log_likelihood

# Calculate scores
ref_scores = calculate_sequence_likelihood(ref_preds["token_logits"], ref_preds["pad_mask"])
var_scores = calculate_sequence_likelihood(var_preds["token_logits"], var_preds["pad_mask"])

# Convert to numpy
ref_scores_np = ref_scores.cpu().numpy()
var_scores_np = var_scores.cpu().numpy()

# Print score shapes for validation
print(f"Reference scores shape: {ref_scores_np.shape}")
print(f"Variant scores shape: {var_scores_np.shape}")